In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying
from lib.network_mapper_OpenDSSnonvec import network_mapper_function as nmf2
from lib.NR3 import NR3_function

Vectorized Solver

In [12]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

#fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
fn = '06node_threephase_unbalanced.dss'
#fn = 'compare05node.dss'

# Time-steps
times = np.linspace(0, 2*np.pi, 5)
# DER changes over time-steps
der = [0, 0, 0, 0, 0]
# Capacitance changes over time-steps
capacitance = [0, 0, 0, 0, 0]

dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")


# Pre-generate matrices
X, g_SB, b_SB, G_KVL, b_KVL, H, g, b = basematrices(fn, slackidx, Vslack, None, None)

for i in range(len(times)):
    # Run NR3 with variations in time, DER, and capacitance
    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i])    
    VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
    INR01[i, :, :] = np.reshape(INR, (3, nline))
    STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR01[i, :, :] = np.reshape(sNR, (3, nnode))


Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
VNR
[[ 1.        +0.j          0.99859589+0.00272071j  0.99719179+0.00544142j
   0.99451947+0.00918927j  0.99575168+0.00922125j  0.99106618+0.01131655j
   0.        +0.j        ]
 [-0.5       -0.8660254j  -0.49831184-0.86822974j -0.49662368-0.87043408j
  -0.49478279-0.87411542j  0.        +0.j         -0.49592268-0.87573023j
  -0.49581689-0.8752546j ]
 [-0.5       +0.8660254j  -0.502981  +0.86336093j -0.505962  +0.86069646j
  -0.51053243+0.85634228j -0.51054702+0.85368992j  0.        +0.j
   0.        +0.j        ]]
INR:
[[ 4.53563300e-01-0.1015113j   4.53563300e-01-0.1015113j
   3.53147875e-01-0.07698884j  1.00650516e-01-0.02417458j
   1.51908420e-01-0.04871614j -7.62938040e-31+0.j        ]
 [-2.32545196e-01-0.26861821j -2.32545196e-01-0.26861821j
  -1.61427112e-01-0.19430923j  1.65750158e-31+0.j
  -1.08052636e-01-0.11011985j -3.74735410e-02-0.03589811j]
 [-1.06257170e-01+0.28693784

Non-vectorized solver reading OpenDSS files

In [13]:
fn= 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
#fn = 'IEEE13_splitmultiphase.dss'
#fn = 'IEEE13_nominalload_vec.dss'
#fn = 'IEEE13_capnominalvoltage.dss'
#fn = 'hello.dss'
#fn = '02node_threephase_unbalanced_unchanged.dss' #original
#fn = '02node_threephase_unbalanced.dss' #aPQ = 0.9 aZ = 0.1
#fn = '06node_threephase_unbalanced.dss' #model = 8
fn = '06node_threephase_unbalanced.dss' 
#fn = 'IEEE_13Node_Modified_01.dss'
#fn = 'compare05node.dss'

VNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR02 =  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR02 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR02 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
spah = np.zeros((len(times), 3, nnode), dtype = "complex") #store load values

for i in range(len(times)):
    network1 = nmf2(fn, times[i])

    spah[i, :, :] = network1.loads.spu

    nnode = network1.nodes.nnode
    nline = network1.lines.nline

    network1.cons.wpu = np.zeros((3,nnode))
    network1.vvc.vvcpu = np.zeros((3,nnode))

    network1.loads.aPQ =1.0*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aI = 0.0*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aZ = 0.0*np.ones((3,nnode))*(network1.loads.spu != 0)

    network1.loads.spu = 1*network1.loads.spu

    slackidx = 0
    Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])


    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_function(network1,slackidx,Vslack,None,None,None,None)

    VNR02[i, :, :] = np.reshape(VNR, (3, nnode))
    INR02[i, :, :] = np.reshape(INR, (3, nline))
    STXNR02[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR02[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR02[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR02[i, :, :] = np.reshape(sNR, (3, nnode))
    print('VNR\n', np.round(VNR,decimals=6))
    print('INR\n', np.round(INR,decimals=6))
    print('STXNR\n', np.round(STXNR,decimals=6))
    print('SRXNR\n', np.round(SRXNR,decimals=6))
    print('iNR\n', np.round(iNR,decimals=6))
    print('sNR\n', np.round(sNR,decimals=6))
    print('itercount:', itercount)

    print('|VNR|\n', np.round(np.abs(VNR),decimals=6))
    print('<VNR\n', np.round(180/np.pi*np.angle(VNR),decimals=6))
    print('D<VNR\n', 180/np.pi*np.angle(VNR) - 180/np.pi*np.angle(VNR[:,[0]]))

BASE
Vbase: 2771.2812921102036
Sbase: 1000000.0
Ibase: 360.8439182435161
Zbase: 7.68

NODES
nnode: 7
nodelist: ['sourcebus', 'a01', 'a02', 'a03', 'a04', 'a05', 'a06']
phases: ['abc', 'abc', 'abc', 'abc', 'ac', 'ab', 'b']
PH:
 [[1 1 1 1 1 1 0]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 0 0]]
LINES
nline: 6
TXnode: ['sourcebus', 'a01', 'a02', 'a03', 'a03', 'a05']
RXnode: ['a01', 'a02', 'a03', 'a04', 'a05', 'a06']
TXnum: [0 1 2 3 3 5]
RXnum: [1 2 3 4 5 6]
phases: ['abc', 'abc', 'abc', 'ac', 'ab', 'b']
PH:
 [[1 1 1 1 1 0]
 [1 1 1 0 1 1]
 [1 1 1 1 0 0]]
config: ['sub_a01', 'a01_a02', 'a02_a03', 'a03_a04', 'a03_a05', 'a05_a06']
length: [2. 2. 2. 2. 2. 2.]

NODES + LINES
inlines:
 [[-1  0  1  2  3  4  5]]
innodes:
 [[-1  0  1  2  3  3  5]]
outlines:
 [[ 0  1  2  3 -1  5 -1]
 [-1 -1 -1  4 -1 -1 -1]]
outnodes:
 [[ 1  2  3  4 -1  6 -1]
 [-1 -1 -1  5 -1 -1 -1]]

CONFIGS
nconf: 6
conflist: ['sub_a01', 'a01_a02', 'a02_a03', 'a03_a04', 'a03_a05', 'a05_a06']
sub_a01 - FZpl:
 [[0.00195312+0.00130208j 0.00065104+0.0

 [[ 1.      +0.j        0.998572+0.002776j  0.997145+0.005553j
   0.994428+0.009379j  0.995685+0.009412j  0.990909+0.01155j
   0.      +0.j      ]
 [-0.5     -0.866025j -0.498278-0.868276j -0.496556-0.870527j
  -0.494678-0.874287j  0.      +0.j       -0.495838-0.875934j
  -0.495728-0.875449j]
 [-0.5     +0.866025j -0.50304 +0.863311j -0.50608 +0.860597j
  -0.510741+0.856163j -0.510755+0.853458j  0.      +0.j
   0.      +0.j      ]]
INR
 [[ 0.462527-0.10382j   0.462527-0.10382j   0.360135-0.078827j
   0.102636-0.024631j  0.154923-0.049642j -0.      +0.j      ]
 [-0.23755 -0.273607j -0.23755 -0.273607j -0.165048-0.197837j
   0.      +0.j       -0.110307-0.112188j -0.038367-0.036508j]
 [-0.108079+0.292756j -0.108079+0.292756j -0.078319+0.19178j
  -0.030651+0.101125j -0.      +0.j        0.      +0.j      ]]
STXNR
 [[ 0.462527+0.10382j   0.461578+0.104956j  0.358669+0.080601j
   0.101833+0.025456j  0.153595+0.050819j -0.      +0.j      ]
 [ 0.355726+0.06892j   0.355932+0.069926j  0.254178+

OpenDSS Solution

In [14]:
dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
#dss.run_command('Redirect 02node_threephase_unbalanced_unchanged.dss')
#dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
#dss.run_command('Redirect IEEE13_splitmultiphase.dss')
#dss.run_command('Redirect 06node_threephase_unbalanced_edit.dss')
#dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
#dss.run_command('Redirect compare05node.dss')

VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
load_arr = np.zeros((len(times), 3, nnode), dtype='complex')
kW_list = np.array([])
kvar_list = np.array([])

# Store the loads
for k in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[k])
    kW_list = np.append(kW_list, dss.Loads.kW())
    kvar_list = np.append(kvar_list, dss.Loads.kvar())

for i in range(len(times)):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
    Vbase = dss.Bus.kVBase() * 1000
    Sbase = 1000000.0
    Ibase = Sbase/Vbase
    Zbase = Vbase/Ibase
    
    # Set slack bus (sourcebus) voltage reference in p.u.
    SlackBusVoltage = 1.000
    dss.Vsources.PU(SlackBusVoltage)
    
    # Time-varying load
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        dss.Loads.kW(kW_list[k]* (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
        dss.Loads.kvar(kvar_list[k] * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
      
    dss.Solution.Convergence(0.000000000001)
    
    # Solve power flow with OpenDSS file
    dss.Solution.Solve()
    if not dss.Solution.Converged:
        print('Initial Solution Not Converged. Check Model for Convergence')
    else:
        print('Initial Model Converged. Proceeding to Next Step.')
        #Doing this solve command is required for GridPV, that is why the monitors
        #go under a reset process
        dss.Monitors.ResetAll()

        # set solution Params
        #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
        dss.Solution.Mode(1)
        dss.Solution.Number(1)
        dss.Solution.StepSize(1)
        dss.Solution.ControlMode(-1)
        dss.Solution.MaxControlIterations(1000000)
        dss.Solution.MaxIterations(30000)
        # Easy process to get all names and count of loads, a trick to avoid
        # some more lines of code
        TotalLoads=dss.Loads.Count()
        AllLoadNames=dss.Loads.AllNames()
        print('OpenDSS Model Compliation Done.')
        print('Iterations: ', dss.Solution.Iterations())
        print('Tolerance: ', dss.Solution.Convergence())

    print('')
    
    
    for k in range(len(dss.Loads.AllNames())):
        dss.Circuit.SetActiveElement('Load.'+dss.Loads.AllNames()[k])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0]) #bus name for load
        idxbs = dss.Circuit.AllBusNames().index(load_bus[0]) #what index is bus at in allbusnames()
        load_ph_arr_temp = [0, 0, 0] #ph of load
        for ph1 in range(1, 4): #update existing ph of load
            pattern = r"\.%s" % (str(ph1))
            load_ph = re.findall(pattern, dss.CktElement.BusNames()[0])
            if load_ph:
                load_ph_arr_temp[ph1 - 1] = 1
        n= 0
        for m in range(len(load_ph_arr_temp)):
            if load_ph_arr_temp[m] == 1:
                #print('squirrel')
                load_arr[i, m, idxbs] +=  (dss.CktElement.Powers()[n] + 1j * dss.CktElement.Powers()[n + 1])*1e3/1e6     #where 1e6 is sbase 
                n += 2
    
    # print(dss.Solution.Converged())

    # Print number of buses, and bus names
    print(len(dss.Circuit.AllBusNames()))
    print(dss.Circuit.AllBusNames())

    # Print number of loads, and load names
    print(len(dss.Loads.AllNames()))
    print(dss.Loads.AllNames())

    print('')

    VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

    for k1 in range(len(dss.Circuit.AllBusNames())):
        
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        
    #     print(dss.Circuit.AllBusNames()[k1])
    #     print(dss.Bus.Nodes())

    #     print('puVOTLAGES - LN CARTESIAN')
    #     print(dss.Bus.PuVoltage())

        ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
        
        Vtemp = np.asarray(dss.Bus.PuVoltage())

        Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
        

    #     print(np.asarray(dss.Bus.Nodes(),'int'))

        VDSS[ph,k1] = Vtemp    

    #     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


    #     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
    #     for k2 in range(len(dss.Bus.Nodes())):
    #         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

    print('VDSS\n', np.round(VDSS,decimals=6))


    IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
        Imn = np.asarray(dss.CktElement.Currents())/Ibase
    #     print(Imn)
        Imn = Imn[0:int(len(Imn)/2)]
    #     print(Imn)
        Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
    #     print(Imn)
        IDSS[ph,k1] = Imn
    #     print('')

    print('IDSS\n', np.round(IDSS,decimals=6))

    STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
    SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
    #     print(dss.CktElement.BusNames())
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    #     print(ph)
        Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)
    #     print(Sk)

    #     print(Sk[0:int(len(Sk)/2)])
    #     print(Sk[int(len(Sk)/2):])

        STXtemp = Sk[0:int(len(Sk)/2)]
        SRXtemp = Sk[int(len(Sk)/2):]

        STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
    #     print(STXtemp)

        SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])
    #     print(SRXtemp)

        STXDSS[ph,k1] = STXtemp
        SRXDSS[ph,k1] = SRXtemp

    VDSS0[i, :, :] = VDSS
    IDSS0[i, :, :] = IDSS
    SRXDSS0[i, :, :] = SRXDSS
    STXDSS0[i, :, :] = STXDSS
    
    
    print('STXDSS\n', np.round(STXDSS,decimals=6))
    print('SRXDSS\n', np.round(SRXDSS,decimals=6))

    print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
    print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
    print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
Iterations:  6
Tolerance:  1e-12

7
['sourcebus', 'a01', 'a02', 'a03', 'a04', 'a05', 'a06']
9
['load_a02_a_01', 'load_a02_b_01', 'load_a02_c_01', 'load_a03_abc_01', 'load_a04_a_01', 'load_a04_c_01', 'load_a05_ab_01', 'load_a05_a_01', 'load_a06_b_01']

VDSS
 [[ 0.999997-6.00000e-06j  0.99859 +2.71100e-03j  0.997184+5.42900e-03j
   0.994507+9.17100e-03j  0.995739+9.20300e-03j  0.991052+1.12960e-02j
   0.      +0.00000e+00j]
 [-0.500003-8.66021e-01j -0.498318-8.68226e-01j -0.496633-8.70431e-01j
  -0.494798-8.74114e-01j  0.      +0.00000e+00j -0.495939-8.75729e-01j
  -0.495835-8.75253e-01j]
 [-0.499995+8.66026e-01j -0.50298 +8.63358e-01j -0.505965+8.60691e-01j
  -0.510544+8.56330e-01j -0.510559+8.53678e-01j  0.      +0.00000e+00j
   0.      +0.00000e+00j]]
IDSS
 [[ 0.453568-0.101645j  0.453568-0.101664j  0.353153-0.07716j
   0.100654-0.024159j  0.151906-0.048702j  0.      +0.j      ]
 [-0.232211-0.268808j -0.

Comparing results between three solutions - DSS, 01 (vectorized), 02 (nonvectorized)

In [15]:
for i in range(len(times)):
    print("Timestep %d \n" % i)
    print('Complex Voltage difference')
    print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 
    print(np.max(np.abs(VDSS0[i] - VNR02[i])))

    print('Complex Current difference')
    print(np.max(np.abs(INR01[i] - INR02[i])))
    print(np.max(np.abs(IDSS0[i] - INR01[i])))
    print(np.max(np.abs(IDSS0[i] - INR02[i])))

    print('Complex TX Power difference')
    print(np.max(np.abs(STXNR01[i] - STXNR02[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR02[i])))

    print('Complex RX Power difference')
    print(np.max(np.abs(SRXNR01[i] - SRXNR02[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR02[i])))

    print('Voltage Magnitude difference')
    print(np.max(np.abs(np.abs(VNR01[i]) - np.abs(VNR02[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR02[i]))))

    print('Voltage Angle difference')
    print(np.max(np.abs(180/np.pi*np.angle(VNR01[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print("\n")

Timestep 0 

Complex Voltage difference
4.577566798522237e-16
2.5143895840152398e-05
2.514389584015669e-05
Complex Current difference
1.1102230246251565e-16
0.00038485759323999793
0.0003848575932400429
Complex TX Power difference
1.2412670766236366e-16
0.00038670812775013034
0.0003867081277501867
Complex RX Power difference
6.938893903907228e-17
0.0003681076775884238
0.000368107677588424
Voltage Magnitude difference
4.440892098500626e-16
1.4467484122149266e-05
1.4467484122149266e-05
Voltage Angle difference
2.842170943040401e-14
0.0011888237606980656
0.0011888237606982877


Timestep 1 

Complex Voltage difference
2.482534153247273e-16
2.5375763315376232e-05
2.537576331537337e-05
Complex Current difference
1.1188630228279524e-16
0.0003879906811097095
0.0003879906811096645
Complex TX Power difference
1.1443916996305594e-16
0.0003898784373624946
0.00038987843736246593
Complex RX Power difference
1.6883057536160649e-16
0.00037129833423243307
0.00037129833423240515
Voltage Magnitude differe

Load comparison between non-vectorized and OpenDSS

In [7]:
for i in range(len(times)):
    print("Timestep %d \n Magnitude of the Difference: \n" % i)
    print(np.abs(spah[i] - load_arr[i]))
    print("Max difference: \n ")
    print(np.max(np.abs(spah[i] - load_arr[i])))
    print("\n")

Timestep 0 
 Magnitude of the Difference: 

[[0.00000000e+00 2.70384930e-11 6.42635962e-12 0.00000000e+00
  0.00000000e+00 5.32391906e-12 3.45714899e-11 0.00000000e+00
  9.79355018e-12 8.12026153e-13 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.36374656e-11 2.68471117e-12 3.77505624e-12
  2.39611839e-12 0.00000000e+00 2.81727322e-12 0.00000000e+00
  0.00000000e+00 1.66024492e-12 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.01786324e-11 3.60895858e-12 0.00000000e+00
  3.11775754e-12 7.25589934e-12 1.72513228e-11 8.79027907e-12
  0.00000000e+00 4.14188786e-12 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]]
Max difference: 
 
3.457148992939218e-11


Timestep 1 
 Magnitude of the Difference: 

[[0.00000000e+00 3.96133662e-13 9.40253225e-14 0.00000000e+00
  0.00000000e+00 6.72506085e-14 5.07442950e-13 0.00000000e+00
  1.43327653e-13 1.18911922e-14 0.00000000

---